In [1]:
import pandas as pd
import plotly.express as px
import plotly.subplots as sp

from pandas import DataFrame
from pandas.core.groupby.generic import DataFrameGroupBy


def standardize(column):
    return (column - column.mean()) / column.std()


def normalize(column):
    return (column - column.min()) / (column.max() - column.min())


def process_group(group: DataFrameGroupBy) -> DataFrame:
    group["punt"] = group["core_boost_clock"] / group["price"]

    return group


# Define euristica weights
CC = 0.2
CB = 0.4
MM = 0.4

# Presentacion PCD

Obtención del DF completo. Primero generamos un dataframe con todas las marcas unidas.


In [2]:
dfs = []
for brand in ["Asus", "Gigabyte", "MSI", "XFX", "Zotac"]:
    df = pd.read_csv(f"../data/{brand}.csv")
    df["brand"] = brand
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True).dropna().reset_index(drop=True)
df = df.drop(columns=["length", "user_score", "user_ratings_count"])

## Gráfico de Rendimiento vs Precio de las marcas por chipset


In [3]:
chipset = "GeForce RTX 4070 SUPER"

# Filter the data for the given chipset
chipset_filtered = df[df["chipset"] == chipset][["price", "core_boost_clock", "brand"]].reset_index(drop=True)
chipset_filtered.head()

,price,core_boost_clock,brand
0,599.99,2550.0,Asus
1,649.00,2595.0,Asus
2,619.95,2550.0,Asus
3,899.95,2550.0,Asus
4,719.99,2670.0,Asus


Podemos ver que todos los valores están presentes, excepto por algunos precios. Eliminaré las filas que no tengan precio.


In [4]:
chipset_filtered = chipset_filtered[["price", "core_boost_clock", "brand"]].dropna()
chipset_filtered.head()

,price,core_boost_clock,brand
0,599.99,2550.0,Asus
1,649.00,2595.0,Asus
2,619.95,2550.0,Asus
3,899.95,2550.0,Asus
4,719.99,2670.0,Asus


In [5]:
chipset_filtered.loc[:, "price"] = standardize(chipset_filtered["price"])
chipset_filtered.loc[:, "core_boost_clock"] = standardize(chipset_filtered["core_boost_clock"])
chipset_filtered.head()

,price,core_boost_clock,brand
0,-0.620380,-0.140515,Asus
1,-0.051001,0.730678,Asus
2,-0.388493,-0.140515,Asus
3,2.864435,-0.140515,Asus
4,0.773732,2.182666,Asus


Finalmente, podemos calcular la puntuación y graficar.


In [6]:
chipset_filtered["punt"] = chipset_filtered["core_boost_clock"] / chipset_filtered["price"]

# Calculamos la media de la puntuación por marca
grouped = chipset_filtered.groupby("brand").mean().sort_values("punt", ascending=False)

fig = px.bar(
    grouped,
    x=grouped.index,
    y="punt",
    title=f"Puntuation of {chipset}",
    labels={"punt": "Puntuation", "index": "Brand"},
    width=800,
    height=600
)
fig.show()

## Gráfico de correlaciones

Se filtra en correlaciones de las gráficas de NVIDIA series 4000, y de AMD series 7000 (las últimas generaciones de tarjetas gráficas).


In [7]:
nvidia = df[df["chipset"].str.contains("RTX 40", case=False, na=False)]
amd = df[df["chipset"].str.contains("RX 7", case=False, na=False)]

column = ["price", "memory", "core_clock", "core_boost_clock"]
nvidia = nvidia[column].corr()
amd = amd[column].corr()

nvidia_plot = px.imshow(
    nvidia,
    text_auto=True,
    title="Correlation Heatmap of GPU Attributes",
    labels={"color": "Correlation"},
    x=column,
    y=column,
    color_continuous_scale="Viridis",
    width=700,
    height=700
)

amd_plot = px.imshow(
    amd,
    text_auto=True,
    title="Correlation Heatmap of GPU Attributes",
    labels={"color": "Correlation"},
    x=column,
    y=column,
    color_continuous_scale="Viridis",
    width=700,
    height=700
)

# Create a subplot
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=["NVIDIA", "AMD"])
fig.add_trace(nvidia_plot.data[0], row=1, col=1)
fig.add_trace(amd_plot.data[0], row=1, col=2)

fig.update_layout(title="Correlation Heatmap of GPU Attributes", width=1500, height=700)
fig.show()

## ¿Qué es mejor? ¿NVIDIA o AMD?

Haremos un gráfico de líneas donde comparamos el rendimiento de las últimas dos generaciones de las tarjetas gráficas de NVIDIA y AMD. Así podremos ver cuál es mejor en términos de rendimiento, además de poder tener un punto de referencia para comparar la calidad de las tarjetas gráficas de ambas marcas.


In [8]:
nvidia4000 = df[df["chipset"].str.contains("RTX 40", case=False, na=False)]
nvidia3000 = df[df["chipset"].str.contains("RTX 30", case=False, na=False)]
amd7000 = df[df["chipset"].str.contains("RX 7", case=False, na=False)]
amd6000 = df[df["chipset"].str.contains("RX 6", case=False, na=False)]

nvidia = pd.concat([nvidia4000, nvidia3000], ignore_index=True).reset_index(drop=True)
amd = pd.concat([amd7000, amd6000], ignore_index=True).reset_index(drop=True)

In [9]:
nvidia.head()

,name,parent_brand,price,memory,core_clock,core_boost_clock,chipset,brand
0,Asus ROG STRIX GAMING OC,NVIDIA,2179.99,24576.0,2235.0,2640.0,GeForce RTX 4090,Asus
1,Asus TUF GAMING,NVIDIA,999.99,16384.0,2210.0,2580.0,GeForce RTX 4080 SUPER,Asus
2,Asus ROG STRIX GAMING OC,NVIDIA,2249.99,24576.0,2235.0,2640.0,GeForce RTX 4090,Asus
3,Asus TUF GAMING OC,NVIDIA,849.00,16384.0,2340.0,2670.0,GeForce RTX 4070 Ti SUPER,Asus
4,Asus ProArt OC,NVIDIA,1149.97,16384.0,2210.0,2640.0,GeForce RTX 4080 SUPER,Asus


In [10]:
amd.head()

,name,parent_brand,price,memory,core_clock,core_boost_clock,chipset,brand
0,Asus DUAL OC,AMD,509.99,16384.0,1295.0,2520.0,Radeon RX 7800 XT,Asus
1,Asus DUAL OC,AMD,439.99,12288.0,1700.0,2599.0,Radeon RX 7700 XT,Asus
2,Asus TUF GAMING OC,AMD,1151.90,24576.0,2300.0,2615.0,Radeon RX 7900 XTX,Asus
3,Asus DUAL OC,AMD,339.97,16384.0,1720.0,2801.0,Radeon RX 7600 XT,Asus
4,Asus TUF GAMING OC,AMD,697.62,16384.0,1295.0,2565.0,Radeon RX 7800 XT,Asus


Ahora, nos quedamos con las columnas que vamos a usar, chipset, price, memory, core_clock y core_boost_clock. Y nos quitamos los que tengan valores nulos.


In [11]:
cols = ["chipset", "price", "core_clock", "core_boost_clock", "memory"]
nvidia = nvidia[cols].dropna()
amd = amd[cols].dropna()

Agrupamos por chipset, estandarizamos valores, calculamos el mismo puntaje de antes, y nos quedamos con el mejor chipeset de cada marca.


In [12]:
nvidia_grouped: DataFrame = nvidia.groupby("chipset").apply(process_group).reset_index(drop=True)
amd_grouped: DataFrame = amd.groupby("chipset").apply(process_group).reset_index(drop=True)

nvidia_max = nvidia_grouped.groupby("chipset")["punt"].max().reset_index()
amd_max = amd_grouped.groupby("chipset")["punt"].max().reset_index()

nvidia_max = pd.merge(nvidia_max, nvidia_grouped, on=["chipset", "punt"], how="inner").drop_duplicates()
amd_max = pd.merge(amd_max, amd_grouped, on=["chipset", "punt"], how="inner").drop_duplicates()

nvidia_max

/tmp/ipykernel_11590/1530778089.py:1: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

/tmp/ipykernel_11590/1530778089.py:2: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,chipset,punt,price,core_clock,core_boost_clock,memory
0,GeForce RTX 3050 6GB,9.325583,159.99,1040.0,1492.0,6144.0
2,GeForce RTX 3050 8GB,9.035452,199.99,1550.0,1807.0,8192.0
3,GeForce RTX 3060 12GB,6.400229,279.99,1320.0,1792.0,12288.0
5,GeForce RTX 3060 8GB,5.949951,303.70,1320.0,1807.0,8192.0
6,GeForce RTX 3060 Ti,3.877437,448.75,1410.0,1740.0,8192.0
7,GeForce RTX 3060 Ti LHR,4.682213,381.23,1410.0,1785.0,8192.0
8,GeForce RTX 3070,3.643869,506.33,1500.0,1845.0,8192.0
9,GeForce RTX 3070 LHR,3.624985,484.14,1500.0,1755.0,8192.0
10,GeForce RTX 3070 Ti,3.182609,575.00,1575.0,1830.0,8192.0
11,GeForce RTX 3080 10GB,1.755018,999.99,1440.0,1755.0,10240.0


Una vez tenemos el mejor chipset de cada marca, podemos calcular la nueva puntación y graficar.


In [13]:
def calc_punt(df):
    return normalize(CC * df["core_clock"] + CB * df["core_boost_clock"] + MM * df["memory"])


def calc_punt_price(df):
    return normalize((CC * df["core_clock"] + CB * df["core_boost_clock"] + MM * df["memory"]) / df["price"])

In [14]:
joined = pd.concat([nvidia_max, amd_max], ignore_index=True).reset_index(drop=True)

joined["punt"] = calc_punt(joined)
joined = joined.sort_values("punt", ascending=False)

fig = px.scatter(
    joined,
    x="chipset",
    y="punt",
    title="Puntuación de las GPU",
    labels={"punt": "Puntuación", "chipset": "Chipset"},
    width=1200,
    height=1000
)
fig.show()

In [15]:
joined["punt"] = calc_punt_price(joined)
joined = joined.sort_values("punt", ascending=False)

fig = px.scatter(
    joined,
    x="chipset",
    y="punt",
    title="Puntuación de las GPU por precio",
    labels={"punt": "Puntuación", "chipset": "Chipset"},
    width=1200,
    height=1000
)
fig.show()

Ahora veamos la media de puntuaciones de cada marca y generación.


In [16]:
nvidia4000 = nvidia4000[cols].dropna().reset_index(drop=True)
nvidia3000 = nvidia3000[cols].dropna().reset_index(drop=True)
amd7000 = amd7000[cols].dropna().reset_index(drop=True)
amd6000 = amd6000[cols].dropna().reset_index(drop=True)

nvidia4000["punt_price"] = calc_punt_price(nvidia4000)
nvidia3000["punt_price"] = calc_punt_price(nvidia3000)
amd7000["punt_price"] = calc_punt_price(amd7000)
amd6000["punt_price"] = calc_punt_price(amd6000)
nvidia4000["punt"] = calc_punt(nvidia4000)
nvidia3000["punt"] = calc_punt(nvidia3000)
amd7000["punt"] = calc_punt(amd7000)
amd6000["punt"] = calc_punt(amd6000)

In [30]:
punt_4000 = (nvidia4000["punt"].mean(), nvidia4000["punt_price"].mean())
punt_3000 = (nvidia3000["punt"].mean(), nvidia3000["punt_price"].mean())
punt_7000 = (amd7000["punt"].mean(), amd7000["punt_price"].mean())
punt_6000 = (amd6000["punt"].mean(), amd6000["punt_price"].mean())

string = f"""
Gen RTX 4000 avg punt: {punt_4000[0]:.3f} avg punt/price: {punt_4000[1]:.3f}
Gen RTX 3000 avg punt: {punt_3000[0]:.3f} avg punt/price: {punt_3000[1]:.3f}
Gen RX 7000 avg punt: {punt_7000[0]:.3f} avg punt/price: {punt_7000[1]:.3f}
Gen RX 6000 avg punt: {punt_6000[0]:.3f} avg punt/price: {punt_6000[1]:.3f}
"""
print(string)


Gen RTX 4000 avg punt: 0.412 avg punt/price: 0.395
Gen RTX 3000 avg punt: 0.272 avg punt/price: 0.361
Gen RX 7000 avg punt: 0.505 avg punt/price: 0.446
Gen RX 6000 avg punt: 0.566 avg punt/price: 0.468



Agrupamos por generación y marca, y calculamos la media de puntuaciones.


In [31]:
nvidia = pd.concat([nvidia4000, nvidia3000], ignore_index=True).reset_index(drop=True)
amd = pd.concat([amd7000, amd6000], ignore_index=True).reset_index(drop=True)

nvidia["punt"] = calc_punt(nvidia)
amd["punt"] = calc_punt(amd)
nvidia["punt_price"] = calc_punt_price(nvidia)
amd["punt_price"] = calc_punt_price(amd)

string = f"""
NVIDIA avg punt: {nvidia["punt"].mean():.3f} avg punt/price: {nvidia["punt_price"].mean():.3f}
AMD avg punt: {amd["punt"].mean():.3f} avg punt/price: {amd["punt_price"].mean():.3f}
"""
print(string)


NVIDIA avg punt: 0.415 avg punt/price: 0.350
AMD avg punt: 0.454 avg punt/price: 0.531



Así, podemos ver que AMD tiene una puntuación media más alta que NVIDIA en todos los aspectos. Sin embargo, algo que no se está considerando es el hecho de que NVIDIA tiene tecnologías exclusivas que pueden mejorar la calidad de la imagen, como DLSS y Ray Tracing, además de tener mejor soporte para juegos AAA, y tener CUDA, que es una tecnología que permite a los desarrolladores de software aprovechar la potencia de las tarjetas gráficas de NVIDIA para acelerar sus aplicaciones.

Se puede ver, además, que AMD tiende a poner precios más bajos a sus tarjetas gráficas de gama más baja. Esto se ve reflejado en el gráfico de puntuaciones en el que tomamos en cuenta el precio. Además, en el gráfico de correlaciones, podemos ver que hay una correlación más fuerte entre el precio y el core_clock en AMD que en NVIDIA, lo puede indicar que AMD tiene que poner sus precios de acuerdo a la frecuencia de reloj de sus tarjetas gráficas para poder competir con NVIDIA, que puede poner sus precios más altos gracias a las tecnologías mencionadas anterior, y por eso en este análisis AMD tiene una puntuación media más alta que NVIDIA.
